<a href="https://colab.research.google.com/github/kaz2ngt/colaboratory/blob/master/8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# リストPART2

In [0]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator

# 8_1: MNISTデータセットの読み込みと前処理

In [4]:
from tensorflow.python.keras.datasets import mnist

(x_train, _), (x_test, _) = mnist.load_data()
# CNNで扱いやすい形に変換
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
# 画像を0-1の範囲に正規化
x_train = x_train/255.
x_test = x_test/255.

11493376/11490434 [==============================] - 0s 0us/step


# 8_2: マスキングノイズを加えて、疑似ノイズデータを生成

In [0]:
def make_masking_noise_data(data_x, percent=0.1):
  size = data_x.shape
  masking = np.random.binomial(n=1, p=percent, size=size)
  return data_x*masking

x_train_masked = make_masking_noise_data(x_train)
x_test_masked = make_masking_noise_data(x_test)

# 8_3: ガウシアンノイズを加えて、疑似ノイズデータを生成

In [0]:
def make_gaussian_noise_data(data_x, scale=0.8):
  gaussian_data_x = data_x + np.random.normal(loc=0, scale=scale, size=data_x.shape)
  gaussian_data_x = np.clip(gaussian_data_x, 0, 1) 
  return gaussian_data_x

x_train_gauss = make_gaussian_noise_data(x_train)
x_test_gauss = make_gaussian_noise_data(x_test)

# 8_4: ノイズを加えた画像とオリジナル画像との比較

In [7]:
from IPython.display import display_png

display_png(array_to_img(x_train[0]))
display_png(array_to_img(x_train_gauss[0]))
display_png(array_to_img(x_train_masked[0]))

# 8_5: CAEモデルの構築

In [8]:
autoencoder = Sequential()

# Encoder箇所
autoencoder.add(
    Conv2D(
        16,
        (3, 3),
        1,
        activation='relu',
        padding='same',
        input_shape=(28, 28, 1)
    )
)
autoencoder.add(
    MaxPooling2D(
        (2, 2),
        padding='same'
    )
)
autoencoder.add(
    Conv2D(
        8,
        (3, 3),
        1,
        activation='relu',
        padding='same',
    )
)
autoencoder.add(
    MaxPooling2D(
        (2, 2),
        padding='same'
    )
)

# Decoder箇所
autoencoder.add(
    Conv2D(
        8,
        (3, 3),
        1,
        activation='relu',
        padding='same',
    )
)
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(
    Conv2D(
        16,
        (3, 3),
        1,
        activation='relu',
        padding='same',
    )
)
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(
    Conv2D(
        1,
        (3, 3),
        1,
        activation='sigmoid',
        padding='same',
    )
)

autoencoder.compile(
    optimizer='adam',
    loss='binary_crossentropy'
)
initial_weights = autoencoder.get_weights()

W0618 08:40:29.121377 139667616905088 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0618 08:40:29.263159 139667616905088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# 8_6: モデルのサマリを確認

In [9]:
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 16)        1

# 8_7: ガウシアンノイズデータを用いて学習

In [11]:
autoencoder.fit(
    x_train_gauss, # 入力：ガウシアンノイズデータ
    x_train, # 正解：オリジナル画像
    epochs=10, # 学習するエポック回数
    batch_size=20, # バッチサイズ
    shuffle=True # シャッフルデータにする
)

Epoch 1/10
60000/60000 [==============================] - 15s 245us/sample - loss: 0.1800
Epoch 2/10
60000/60000 [==============================] - 12s 197us/sample - loss: 0.1568
Epoch 3/10
60000/60000 [==============================] - 12s 197us/sample - loss: 0.1521
Epoch 4/10
60000/60000 [==============================] - 12s 196us/sample - loss: 0.1495
Epoch 5/10
60000/60000 [==============================] - 12s 201us/sample - loss: 0.1478
Epoch 6/10
60000/60000 [==============================] - 12s 198us/sample - loss: 0.1466
Epoch 7/10
60000/60000 [==============================] - 12s 194us/sample - loss: 0.1459
Epoch 8/10
60000/60000 [==============================] - 12s 194us/sample - loss: 0.1453
Epoch 9/10
60000/60000 [==============================] - 12s 194us/sample - loss: 0.1448
Epoch 10/10
60000/60000 [==============================] - 12s 193us/sample - loss: 0.1444


# 8_8: ガウシアンノイズデータで学習したモデルの予測

In [0]:
gauss_preds = autoencoder.predict(x_test_gauss)

# 8_9: ガウシアンノイズ画像、CAEモデル、オリジナル画像の予測の比較

In [14]:
for i in range(10):
  display_png(array_to_img(x_test[i]))
  display_png(array_to_img(x_test_gauss[i]))
  display_png(array_to_img(gauss_preds[i]))
  print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------
